## Reconstruct missing residues using Modeller
Links:
- [Comparative protein structure modeling using MODELLER](https://doi.org/10.1002/cpbi.3)

- [Comparative modeling of protein 3D structures using MODELLER](https://salilab.org/modeller/wiki/Missing%20residues)

### Get the sequence from the template (pdb file)

In [ ]:
# Get the sequence of the target PDB file, and write to an alignment file
from modeller import *
code = '2xa0_A'

e = Environ()
m = Model(e, file=code)
aln = Alignment(e)
aln.append_model(m, align_codes=code)
outSeq = "2xa0_A.seq"
aln.write(file=outSeq)
[print (l,end="") for l in open (outSeq).readlines()];

Because either AutoModel or LoopModel will build a comparative model using your input PDB as a template, potentially all of the atoms in your final model could move. If you really don't want the non-missing residues to move, you can override the select_atoms method to select only the missing residues with a script similar to that below (note that the residue numbers are off by 1, since Modeller numbers the model starting at 1 in chain A, while the original PDB started numbering at 2):

In [ ]:
from modeller import *
from modeller.automodel import *    # Load the AutoModel class

log.verbose()
env = Environ()

# directories for input atom files
env.io.atom_files_directory = ['.', '../atom_files']

class MyModel(AutoModel):
    def select_atoms(self):
        return Selection(self.residue_range('1:A', '9:A'),
                         self.residue_range('32:A', '91:A'))

a = MyModel(env, alnfile = '2xa0_A_Alignment.ali',
            knowns = '2xa0_A', sequence = '2xa0_A_fill')
a.starting_model= 1
a.ending_model  = 10

a.make()

We can now use the standard Modeller 'LoopModel' class to generate a model with all residues, and then to refine the loop regions:

In [ ]:
from modeller import *
from modeller.automodel import *    # Load the AutoModel class

log.verbose()
env = Environ()

# directories for input atom files
env.io.atom_files_directory = ['.', '../atom_files']

a = LoopModel(env, alnfile = 'alignment.ali',
              knowns = '1qg8', sequence = '1qg8_fill')
a.starting_model= 1
a.ending_model  = 1

a.loop.starting_model = 1
a.loop.ending_model   = 6
a.loop.md_level       = refine.fast

a.make()